In [40]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import networkx as nx
import ujson as json
import metis
import numpy as np
import random
from matplotlib import pyplot as plt
from pprint import pprint
from collections import Counter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dataset="circular"

In [3]:
g = nx.read_gpickle('data/{}.gpkl'.format(dataset))

In [4]:
print(g.number_of_nodes())

50000


In [5]:
def populate_r_and_c(g, pr, target_nodes, k):
    """
    g: graph
    pr: pagerank scores (dict)
    targetr_nodes: list of nodes to consider
    k: number of highest degree nodes to take
    
    return:
    r: page rank score
    c: node importance on target_nodes in terms of
    """
    r = np.zeros(g.number_of_nodes())    
    node2id = {n: i for i, n in enumerate(g.nodes_iter())}
    for n in g.nodes_iter():
        r[node2id[n]] = pr[n]
    
    # take highest degree nodes
    top_nodes = sorted(target_nodes,
                       key=lambda n: g.degree(n),
                       reverse=True)[:k]
    c = np.zeros(g.number_of_nodes())
    for n in top_nodes:
        c[node2id[n]] = 1
    return r, c
    

In [9]:
def controversy_score(g, top_percent=0.001):
    """consider only two sides only
    top_percent: percentage of high degree nodes to consider for the c vector 
    """
    k = int(g.number_of_nodes() * top_percent)
    assert k > 0
    print('k={}'.format(k))
    cuts, parts = metis.part_graph(g)
    aux = lambda p, target: int(target == p)
    
    # personalization vector
    part_sizes = Counter(parts)
    e_0 = {n: aux(p, 0) / part_sizes[0] for n, p in zip(g.nodes(), parts)}
    e_1 = {n: aux(p, 1) / part_sizes[1] for n, p in zip(g.nodes(), parts)}

    # pagerank scores
    pr0 = nx.pagerank(g, alpha=0.85, personalization=e_0, dangling=e_0, max_iter=10000)
    pr1 = nx.pagerank(g, alpha=0.85, personalization=e_1, dangling=e_1, max_iter=10000)

    # nodes at two sides
    nodes0 = [n for n, p in zip(g.nodes(), parts) if p == 0]
    nodes1 = [n for n, p in zip(g.nodes(), parts) if p == 1]

    r0, c0 = populate_r_and_c(g, pr0, nodes0, k)
    r1, c1 = populate_r_and_c(g, pr1, nodes1, k)
    
    r_list = [r0, r1]
    c_list = [c0, c1]
    controversy, non_controversy = 0, 0
    k = len(r_list)
    for i, r in enumerate(r_list):
        for j, c in enumerate(c_list):
            prod = np.sum(r * c)
            # print(prod, prod * part_sizes[i])
            if i == j:
                controversy += prod
            else:
                non_controversy += prod
    print(controversy, non_controversy)
    return controversy / (controversy + non_controversy)

In [16]:
print(controversy_score(g))

k=50
0.00343846776375 0.000561532236253
0.859616940937


In [8]:
from joblib import Parallel, delayed
datasets = ['beefban', 'ukraine', 'baltimore', 'circular', 'barbell_1000_0', 'barabasi', 'star']
scores = Parallel(n_jobs=4)(delayed(controversy_score)(nx.read_gpickle('data/{}.gpkl'.format(dataset)))
                           for dataset in datasets)


k=28
k=56
k=93
k=50
0.00343846776375 0.000561532236253
k=2
0.387652790091 0.0509276973833
k=50
0.282845642967 0.0388647300526
k=50
0.050028412602 0.0393195924934
0.471745134496 0.471147206782
0.462045118917 0.103348028087
0.00397886921661 2.2829080248e-05


In [12]:
for d, s in sorted(list(zip(datasets, scores)), key=lambda t: t[1], reverse=True):
    print('- {}: {}'.format(d, s))

- barbell_1000_0: 0.9942951520693315
- beefban: 0.8838806129369029
- ukraine: 0.8791934195724288
- circular: 0.8596169409366637
- baltimore: 0.8172103276545415
- barabasi: 0.5599275837061325
- star: 0.5003170710418742


In [41]:
from rwc import controversy_score as f
# print(f(g))

In [42]:
g = nx.star_graphr_graph(1000)

node2cluster = {n: 0 for n in g.nodes()}
f(g, node2cluster)

(0.5, {'pr0': None, 'pr1': None})